# Read the films genres

In [8]:
import pandas as pd

films = pd.read_csv('./dataset/movies.csv')
films.set_index('movieId', inplace=True)
films.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


# Read the Ratings

In [9]:
ratings = pd.read_csv('./dataset/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


<br>
<br>
<br>

# 1. Recomending by Views And Rating

### Extraction the views and the mean rating

In [10]:
view_mean = ratings.groupby(by="movieId").agg({"rating": "mean", "timestamp": "count"})
view_mean.rename(columns={"timestamp": "views"}, inplace=True)
view_mean.sort_values(by="views", inplace=True, ascending=False)
view_mean.head()

,rating,views
movieId,,
356,4.164134,329
318,4.429022,317
296,4.197068,307
593,4.161290,279
2571,4.192446,278


### Top 10 movies

In [11]:
first_recomendation = films.merge(view_mean, on="movieId", how="inner")
first_recomendation.sort_values(by=["views", "rating"], ascending=False, inplace=True)

first_recomendation.head(10)

,title,genres,rating,views
movieId,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.164134,329
318,"Shawshank Redemption, The (1994)",Crime|Drama,4.429022,317
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.197068,307
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.161290,279
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.192446,278
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,4.231076,251
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,3.750000,238
110,Braveheart (1995),Action|Drama|War,4.031646,237
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,3.970982,224


<br>
<br>
<br>

# 2. Same genres watched previously

    The variable bellow represents the movie that user already viewed

In [12]:
USER_WATCHING = "Comedy|Crime"

    Filter according with the history from the user and returns the most visualized movies of the same genres

In [13]:
top_same_genre = first_recomendation[[USER_WATCHING in genre for genre in first_recomendation.genres]]
top_same_genre.head(10)

,title,genres,rating,views
movieId,,,,
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.197068,307
608,Fargo (1996),Comedy|Crime|Drama|Thriller,4.116022,181
367,"Mask, The (1994)",Action|Comedy|Crime|Fantasy,3.184713,157
153,Batman Forever (1995),Action|Adventure|Comedy|Crime,2.916058,137
1732,"Big Lebowski, The (1998)",Comedy|Crime,3.924528,106
778,Trainspotting (1996),Comedy|Crime|Drama,4.039216,102
2987,Who Framed Roger Rabbit? (1988),Adventure|Animation|Children|Comedy|Crime|Fant...,3.572165,97
2502,Office Space (1999),Comedy|Crime,4.090426,94
4027,"O Brother, Where Art Thou? (2000)",Adventure|Comedy|Crime,3.808511,94


<br>
<br>
<br>

## 3. Find the most similar persons using rating

In [19]:
import numpy as np

movies_rating = ratings.merge(pd.DataFrame(films["title"]), on="movieId", how="inner")
movies_rating.drop(columns=["movieId", "timestamp"], inplace=True)
users = movies_rating['userId'].drop_duplicates().values

movies_rating = movies_rating.pivot_table(columns=['title'], index='userId', values='rating')

movies_rating.replace({np.nan: 0}, inplace=True)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Convert in train objects

In [17]:
X = movies_rating.values
y = movies_rating.index.values

#### Modeling

In [47]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)

<br>
<br>

#### Find near peaple

In [157]:
from IPython.display import HTML, display

target = 98

# Find the most similar
distances, indices = model.kneighbors([X[target]])

display(HTML("<h2>Similar people id:</h2> <h4>{}</h4>".format(str(indices[0].tolist())[1:-1])))

<br>
<br>

#### Recomending

In [158]:
mask = ratings.userId.isin(indices[0][1:])

# Filter the reviews by the near people
near_reviews = ratings[mask]

# Group by films
near_reviews = near_reviews.groupby("movieId").agg({"rating": "mean"})

# Remove films already viewed by the user
user_films = ratings[ratings.userId == target].movieId.values

mask = near_reviews.index.isin(user_films) == False
near_reviews = near_reviews[mask]
near_reviews.reset_index(inplace=True)

# Merge the movies
movies_views = films.merge(view_mean, on="movieId", how="inner")[["title", "genres", "views"]]
movies_views = movies_views.merge(near_reviews, on="movieId", how="inner")
recommended_movies = movies_views

# Sort descending by rating
recommended_movies.sort_values(["rating", "views"], inplace=True, ascending=False)

recommended_movies.head(20)

,movieId,title,genres,views,rating
11,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,204,5.0
90,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,200,5.0
59,590,Dances with Wolves (1990),Adventure|Drama|Western,164,5.0
358,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149,5.0
18,165,Die Hard: With a Vengeance (1995),Action|Crime|Thriller,144,5.0
29,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller,133,5.0
146,2329,American History X (1998),Crime|Drama,129,5.0
237,5349,Spider-Man (2002),Action|Adventure|Sci-Fi|Thriller,122,5.0
171,2918,Ferris Bueller's Day Off (1986),Comedy,109,5.0
338,48516,"Departed, The (2006)",Crime|Drama|Thriller,107,5.0
